In [ ]:
import psycopg2
import requests

from config import settings

In [ ]:
conn = psycopg2.connect(**settings.DB_CONN)
cur = conn.cursor()

In [ ]:
start_date = "2019-03-01"
end_date = "2019-03-12"

base_url = "https://moto.data.socrata.com/resource/yqmq-bpqk.json?"
url = f"{base_url}$where=date_trunc_ymd(updated_at) between '{start_date}' and '{end_date}'"

In [ ]:
headers = {"X-App-Token": settings.SOCRATA_API_TOKEN}

In [ ]:
results = requests.get(url, headers=headers)

In [ ]:
results.headers['X-SODA2-Fields']

In [ ]:
results.headers['X-SODA2-Fields'].replace('[', '').replace(']', '').replace('"','').split(',')

In [ ]:
results.json()[0]

In [ ]:
data = results.json()
keys_to_check = ['address_2', 'country', 'zip']
for key in keys_to_check:
    for row in data:
        if not key in row.keys():
            row[key] = None

print(len(data))

In [ ]:
data[0]

In [ ]:
cur.execute("truncate table raw.east_point_incidents;")
conn.commit()

In [ ]:
cur.executemany(settings.INCIDENT_INSERT_SQL, data)
conn.commit()

In [ ]:
cur.execute(settings.INCIDENT_MERGE_SQL)
conn.commit()

In [ ]:
cur.close()
conn.close()